In [1]:
import pandas as pd
import os

In [30]:
root_path = r'C:\Users\ADE17\Desktop\Masters\Projects\AIChallenge_OWL\SollIch-Hackathon_Daten\Data_Participants3'

In [86]:
def load_multiple_data(root_path):
    data_dict = {}
    x_folder = "Train_X"
    y_folder = "Train_Y"

    x_files = os.listdir(os.path.join(root_path, x_folder))
    y_files = os.listdir(os.path.join(root_path, y_folder))

    for x_file, y_file in zip(x_files, y_files):
        if x_file.endswith('.pq') and y_file.endswith('.pq'):
            path_X = os.path.join(x_folder, x_file)
            path_Y = os.path.join(y_folder, y_file)
            df_X = pd.read_parquet(os.path.join(root_path, path_X))
            df_Y = pd.read_parquet(os.path.join(root_path, path_Y))
            target_col_1 = df_X['ProzessData_ActData_AB1_Temperature_DR1_WaterMixingStage']
            targets = pd.concat([df_Y, target_col_1], axis=1)
            df_X = df_X.drop(['ProzessData_ActData_AB1_Temperature_DR1_WaterMixingStage'], axis=1)
            date = x_file.split('_')[1:]  # Extracting month and day
            date_key = '_'.join(date)[:5]  # Creating the 'MM_DD' format
            data_dict[date_key] = {'features': df_X, 'targets': targets}

    return data_dict

In [87]:
data = load_multiple_data(root_path)

In [91]:
data['03_14']['features']

,ProzessData_ActData_AB1_Analogs_DX1_MassPressure,ProzessData_ActData_AB1_Analogs_GY1_MassLevelTank,ProzessData_ActData_AB1_Current_DV1_Scraper,ProzessData_ActData_AB1_Current_DW1_RiserPumpFwd,ProzessData_ActData_AB1_Speed_DV1_Scraper,ProzessData_ActData_AB1_Speed_DW1_RiserPumpFwd,ProzessData_ActData_AB1_Temperature_DP1_MassHeatingStage,ProzessData_ActData_AB1_Temperature_DP1_WaterHeatingStage,ProzessData_ActData_AB1_Temperature_DQ1_MassCoolingStage,ProzessData_ActData_AB1_Temperature_DQ1_WaterCoolingStage,ProzessData_ActData_AB1_Temperature_DS1_WaterPipe,ProzessData_ActData_AB1_Temperature_DU1_WaterTank,ProzessData_ActData_AB1_Temperature_DX1_MassInfeed
2023-03-14 01:00:00+00:00,0.92,63.349998,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,41.849998,42.099998,41.900002
2023-03-14 01:00:01+00:00,0.93,63.340000,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,41.849998,42.099998,41.900002
2023-03-14 01:00:02+00:00,0.93,63.340000,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,41.849998,42.000000,41.900002
2023-03-14 01:00:03+00:00,0.92,63.349998,0.0,0.0,0.0,0.0,41.299999,41.900002,41.700001,42.0,41.849998,42.000000,41.900002
2023-03-14 01:00:04+00:00,0.92,63.340000,0.0,0.0,0.0,0.0,41.299999,41.900002,41.700001,42.0,41.849998,42.000000,41.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-14 19:59:56+00:00,0.80,63.849998,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,41.849998,42.000000,41.900002
2023-03-14 19:59:57+00:00,0.81,63.849998,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,41.849998,42.000000,41.900002
2023-03-14 19:59:58+00:00,0.80,63.849998,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,41.849998,42.000000,41.900002
2023-03-14 19:59:59+00:00,0.82,63.860001,0.0,0.0,0.0,0.0,41.299999,42.000000,41.700001,42.0,41.849998,42.000000,41.900002
